# A4: Word Sense Disambiguation using Neural Networks
Adam Ek

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Write all your answers and the code in the appropriate boxes below.


A problem with static distributional vectors is the difficulty of distinguishing between different *word senses*. We will continue our exploration of word vectors by considering *trainable vectors* or *word embeddings* for Word Sense Disambiguation (WSD).

The goal of word sense disambiguation is to train a model to find the sense of a word (homonyms of a word-form). For example, the word "bank" can mean "sloping land" or "financial institution". 

(a) "I deposited my money in the **bank**" (financial institution)

(b) "I swam from the river **bank**" (sloping land)

In case a) and b) we can determine that the meaning of "bank" based on the *context*. To utilize context in a semantic model we use *contextualized word representations*. Previously we worked with *static word representations*, i.e. the representation does not depend on the context. To illustrate we can consider sentences (a) and (b), the word **bank** would have the same static representation in both sentences, which means that it becomes difficult for us to predict its sense. What we want is to create representations that depend on the context, i.e. *contextualized embeddings*. 

We will create contextualized embeddings with Recurrent Neural Networks. You can read more about recurrent neural netoworks [here](https://colah.github.io/posts/2015-08-Understanding-LSTMs/). Your overall task in this lab is to create a neural network model that can disambiguate the word sense of 30 different words. 

In [ ]:
!pip install nvidia-cudnn-cu11==8.5.0.96

In [21]:
# first we import some packages that we need
import torch
import torch.nn as nn
# import torchtext
from torch.utils.data import DataLoader, Dataset
import pickle
import torch.optim as optim
import pandas as pd
import numpy as np

# our hyperparameters (add more when/if you need them)
device = torch.device('cuda:3')

batch_size = 8
learning_rate = 0.001
epochs = 3

embedding_dims = 128
hidden_dims = 128

# 1. Working with data

A central part of any machine learning system is the data we're working with. In this section we will split the data (the dataset is located here: ``wsd-data/wsd_data.txt``) into a training set and a test set. We will also create a baseline to compare our model against. Finally, we will use TorchText to transform our data (raw text) into a convenient format that our neural network can work with.

## Data

The dataset we will use contain different word sense for 30 different words. The data is organized as follows (values separated by tabs): 
- Column 1: word-sense
- Column 2: word-form
- Column 3: index of word
- Column 4: white-space tokenized context

### Splitting the data

Your first task is to seperate the data into a *training set* and a *test set*. The training set should contain 80% of the examples and the test set the remaining 20%. The examples for the test/training set should be selected **randomly**. Save each dataset into a .csv file for loading later. **[2 marks]**

In [3]:
def data_split(path_to_dataset):
    with open(path_to_dataset) as f:
        lines = [line.split('\t') for line in f]
    df = pd.DataFrame(lines)
    df_test = df.sample(frac=0.2)
    df_train = df.drop(df_test.index)
    df_test.to_csv('test.csv')
    df_train.to_csv('train.csv')
    return
data_split('wsd_data.txt')

### Creating a baseline

Your second task is to create a *baseline* for the task. A baseline is a "reality check" for a model, given a very simple heuristic/algorithmic/model solution to the problem, can our neural network perform better than this?
The baseline you are to create is the "most common sense" (MCS) baseline. For each word form, find the most commonly assigned sense to the word, and label a words with that sense. **[2 marks]**

E.g. In a fictional dataset, "bank" have two senses, "financial institution" which occur 5 times and "side of river" 3 times. Thus, all 8 occurences of bank is labeled "financial institution" and this yields an MCS accuracy of 5/8 = 62.5%. If a model obtain a higher score than this, we can conclude that the model *at least* is better than selecting the most frequent word sense.

In [4]:
def mcs_baseline(data):
    '''Takes data in the form of a pandas DataFrame and returns a dictionay mapping
    from word form to most common word sense.
    '''
    df_grouped = data.groupby('1')
    mapping = {}
    for group in df_grouped:
        mapping[group[0].split('.')[0]] = group[1].mode()['0'][0]
    return mapping  

In [5]:
test = pd.read_csv('test.csv', index_col=0)
mcs_baseline(test)

{'active': 'active%5:00:00:involved:00',
 'bad': 'bad%5:00:00:intense:00',
 'bring': 'bring%2:38:00::',
 'build': 'build%2:36:00::',
 'case': 'case%1:11:00::',
 'common': 'common%3:00:02::',
 'critical': 'critical%3:00:03::',
 'extend': 'extend%2:40:04::',
 'find': 'find%2:40:02::',
 'follow': 'follow%2:40:00::',
 'force': 'force%1:07:02::',
 'hold': 'hold%2:40:04::',
 'keep': 'keep%2:41:03::',
 'lead': 'lead%2:38:01::',
 'life': 'life%1:26:01::',
 'line': 'line%1:04:01::',
 'major': 'major%3:00:06::',
 'national': 'national%3:00:01::',
 'order': 'order%1:14:00::',
 'physical': 'physical%3:00:00::',
 'place': 'place%1:04:01::',
 'point': 'point%1:09:01::',
 'position': 'position%1:26:02::',
 'positive': 'positive%3:00:01::',
 'professional': 'professional%3:01:00::',
 'regular': 'regular%5:00:00:standard:02',
 'security': 'security%1:26:00::',
 'see': 'see%2:31:00::',
 'serve': 'serve%2:42:03::',
 'time': 'time%1:11:00::'}

### Creating data iterators

To train a neural network, we first need to prepare the data. This involves converting words (and labels) to a number, and organizing the data into batches. We also want the ability to shuffle the examples such that they appear in a random order.  

To do all of this we will use the torchtext library (https://torchtext.readthedocs.io/en/latest/index.html). In addition to converting our data into numerical form and creating batches, it will generate a word and label vocabulary, and data iterators than can sort and shuffle the examples. 

Your task is to create a dataloader for the training and test set you created previously. So, how do we go about doing this?

1) First we create a ``Field`` for each of our columns. A field is a function which tokenize the input, keep a dictionary of word-to-numbers, and fix paddings. So, we need four fields, one for the word-sense, one for the position, one for the lemma and one for the context. 

2) After we have our fields, we need to process the data. For this we use the ``TabularDataset`` class. We pass the name and path of the training and test files we created previously, then we assign which field to use in each column. The result is that each column will be processed by the field indicated. So, the context column will be tokenized and processed by the context field and so on. 

3) After we have processed the dataset we need to build the vocabulary, for this we call the function ``build_vocab()`` on the different ``Fields`` with the output from ``TabularDataset`` as input. This looks at our dataset and creates the necessary vocabularies (word-to-number mappings). 

4) Finally, the last step. In the last step we load the data objects given by the ``TabularDataset`` and pass it to the ``BucketIterator`` class. This class will organize our examples into batches and shuffle them around (such that for each epoch the model observe the examples in a different order). When we are done with this we can let our function return the data iterators and vocabularies, then we are ready to train and test our model!

Implement the dataloader. [**2 marks**]

*hint: for TabularDataset and BucketIterator use the class function splits()* 

In [4]:
class Vocab:
    def __init__(self, tokens, pad_token='PAD', unk_token='UNK'):
        self.tokens = tokens+[unk_token]
        self.i2t = {i: t for i, t in enumerate(self.tokens, start=1)}
        self.i2t[0] = pad_token
        self.t2i = {v: k for k, v in self.i2t.items()}
        self.tokens += [pad_token] 
        
    def __len__(self):
        return len(self.tokens)
    
    def __getitem__(self, x):
        if type(x) == str:
            return self.t2i[x]
        if type(x) == int:
            return self.i2t[x]

In [5]:
class WSD_Dataset(Dataset):
    def __init__(self, csv_file,
                 train=True,
                 unk_token='UNK',
                 pad_token='PAD',
                 vocab=None,
                 sense_vocab=None):
        
        self.unk_token = unk_token
        self.pad_token = pad_token
        self.alldata = pd.read_csv(csv_file, index_col=0)
        
        self.senses = [sense for sense in self.alldata['0']]
        self.words = [line.split('.')[0] for line in self.alldata['1']]
        self.indices = [int(x) for x in self.alldata['2']]
        self.contexts = [line.lower().split() for line in self.alldata['3']]
        
        if train:
            self.tokens = list(set([token for line in self.contexts for token in line]))
            self.vocab = Vocab(self.tokens, pad_token=self.pad_token, unk_token=self.unk_token)    
            self.sense_vocab = Vocab(list(set(self.senses)))
            self.int_contexts = [[self.vocab[word] for word in seq] for seq in self.contexts]
        
        else:
            self.vocab = vocab
            self.sense_vocab = sense_vocab
            self.int_contexts = [[self.vocab[word] if word in self.vocab.t2i else self.vocab[self.unk_token] for word in seq] for seq in self.contexts]
            
        self.int_senses = [self.sense_vocab[sense] for sense in self.senses]
        
    
    def __getitem__(self, idx):
        
        return (self.int_senses[idx], self.indices[idx], self.int_contexts[idx])

    def __len__(self):
        return len(self.int_senses)

In [6]:
def pad_fn(data):
    lengths = [len(x[2]) for x in data]
    max_len = max(lengths)
    padded_data = [(x[0], x[1], [w for w in x[2]+[0]*(max_len-len(x[2]))]) for x in data]
    return padded_data

def data_load(path_to_csv, batch_size, train=True, vocab=None, sense_vocab=None): 
    dataset = WSD_Dataset(path_to_csv, train=train, vocab=vocab, sense_vocab=sense_vocab)
    dataloader = DataLoader(dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            collate_fn=pad_fn)
    if train==True:
        return dataloader, dataset.vocab, dataset.sense_vocab
    else:
        return dataloader

In [7]:
# checking that it works:

dataloader, vocab, sense_vocab = data_load('test.csv', batch_size)
print('pad_index =', vocab.t2i['PAD'])
for i, batch in enumerate(dataloader):
    print('Batch size:', len(batch)) 
    for example in batch:
        print(example)
        print('context length:', len(example[2]))
  # Only look at first batch.
    break

pad_index = 0
Batch size: 8
(131, 131, [28746, 3853, 33124, 8637, 3853, 17633, 4032, 16309, 26458, 5166, 3853, 29311, 22367, 2180, 8602, 12757, 22204, 14671, 26081, 29272, 35159, 3853, 17633, 4032, 25936, 25137, 8602, 12383, 3853, 28977, 22367, 3853, 2539, 8602, 25936, 10181, 17618, 8137, 11394, 30230, 8602, 25936, 5121, 7105, 25936, 17618, 10681, 16461, 6796, 10470, 22204, 1254, 8602, 25936, 13410, 7307, 32542, 24433, 29272, 16173, 2206, 29272, 25936, 23907, 3853, 6357, 29272, 32766, 22367, 19477, 29272, 25936, 3114, 22423, 8602, 2527, 196, 8602, 24076, 3853, 19299, 22367, 10470, 22204, 2787, 7107, 128, 25734, 34566, 11394, 10447, 25936, 24453, 9293, 29272, 11363, 3853, 17633, 4032, 19596, 10681, 10470, 22204, 2787, 2539, 17854, 6271, 22496, 8565, 7105, 8602, 21833, 8602, 15668, 196, 8602, 18627, 3865, 33167, 25936, 3853, 17633, 4032, 22383, 25826, 22204, 8866, 19596, 28551, 8602, 24076, 9649, 9293, 3853, 20941, 27123, 22367, 13206, 8008, 12854, 18627, 17633, 15597, 22367, 3853, 18126

# 2.1 Creating and running a Neural Network for WSD

In this section we will create and run a neural network to predict word senses based on *contextualized representations*.

### Model

We will use a bidirectional Long-Short-Term Memory (LSTM) network to create a representation for the sentences and a Linear classifier to predict the sense of each word.

When we initialize the model, we need a few things:

    1) An embedding layer: a dictionary from which we can obtain word embeddings
    2) A LSTM-module to obtain contextual representations
    3) A classifier that compute scores for each word-sense given *some* input


The general procedure is the following:

    1) For each word in the sentence, obtain word embeddings
    2) Run the embedded sentences through the RNN
    3) Select the appropriate hidden state
    4) Predict the word-sense 

**Suggestion for efficiency:**  *Use a low dimensionality (32) for word embeddings and the LSTM when developing and testing the code, then scale up when running the full training/tests*
    
Your tasks will be to create two different models (both follow the two outlines described above), described below:

In the first approach to WSD, you are to select the index of our target word (column 3 in the dataset) and predict the word sense. **[5 marks]**


In [8]:
from IPython import embed

In [22]:
class WSDModel_approach1(nn.Module):
    def __init__(self, vocab_dims, embedding_dims, hidden_dims, n_senses, pad_index=0):
        super(WSDModel_approach1, self).__init__()
        
        self.pad_index=pad_index
        self.embeddings = nn.Embedding(num_embeddings=vocab_dims, 
                                               embedding_dim=embedding_dims,
                                               padding_idx=pad_index)
        self.rnn = nn.LSTM(embedding_dims, hidden_dims, batch_first=True)
        self.classifier = nn.Linear(hidden_dims, n_senses)
    
    def forward(self, batch):
        embedded_seq = self.embeddings(batch)
        output, (h, c) = self.rnn(embedded_seq)
        predictions = self.classifier(output)
        # raise embed()        
        return predictions

In the second approach to WSD, you are to predict the word sense based on the final hidden state given by the RNN. **[5 marks]**

In [23]:
class WSDModel_approach2(nn.Module):
    def __init__(self, vocab_dims, embedding_dims, hidden_dims, n_senses, pad_index=0):
        super(WSDModel_approach2, self).__init__()
        
        self.pad_index=pad_index
        self.embeddings = nn.Embedding(num_embeddings=vocab_dims, 
                                               embedding_dim=embedding_dims,
                                               padding_idx=pad_index)
        self.rnn = nn.LSTM(embedding_dims, hidden_dims, batch_first=True)
        self.classifier = nn.Linear(hidden_dims, n_senses)
    
    def forward(self, batch):
        embedded_seq = self.embeddings(batch)
        output, (h, c) = self.rnn(embedded_seq)
        predictions = self.classifier(h)
        # raise embed()
        return predictions

### Training and testing the model

Now we are ready to train and test our model. What we need now is a loss function, an optimizer, and our data. 

- First, create the loss function and the optimizer.
- Next, we iterate over the number of epochs (i.e. how many times we let the model see our data). 
- For each epoch, iterate over the dataset (``train_iter``) to obtain batches. Use the batch as input to the model, and let the model output scores for the different word senses.
- For each model output, calculate the loss (and print the loss) on the output and update the model parameters.
- Reset the gradients and repeat.
- After all epochs are done, test your trained model on the test set (``test_iter``) and calculate the total and per-word-form accuracy of your model.

Implement the training and testing of the model **[4 marks]**

**Suggestion for efficiency:** *when developing your model, try training and testing the model on one or two batches (for each epoch) of data to make sure everything works! It's very annoying if you train for N epochs to find out that something went wrong when testing the model, or to find that something goes wrong when moving from epoch 0 to epoch 1.*

In [11]:
train_iter, vocab, labels = data_load('train.csv', batch_size)
test_iter = data_load('test.csv', batch_size, train=False, vocab=vocab, sense_vocab=labels)

In [12]:
# Check that it works for the test set too:

print('pad_index =', vocab.t2i['PAD'])
print('unk_index =', vocab.t2i['UNK'])
for i, batch in enumerate(test_iter):
    print('Batch size:', len(batch)) 
    for example in batch:
        print(example)
        print('context length:', len(example[2]))
        print('unk in context:', vocab.t2i['UNK'] in example[2])
  # Only look at first batch.
    break

pad_index = 0
unk_index = 70401
Batch size: 8
(61, 73, [7653, 3465, 57653, 12999, 17157, 64675, 19843, 58067, 26356, 44643, 17157, 48328, 12193, 44624, 37379, 51350, 61196, 69659, 24971, 21315, 51408, 38005, 15965, 7653, 59872, 47322, 47550, 69654, 59902, 64675, 7653, 49033, 44182, 17618, 44417, 17094, 36969, 35885, 43589, 54762, 1491, 21315, 44307, 58875, 7653, 53346, 44182, 7653, 9503, 44182, 7653, 59872, 35994, 17157, 58067, 44624, 4832, 49680, 18529, 19584, 43552, 51350, 47968, 69659, 6706, 59398, 58067, 62018, 51350, 12554, 69468, 47550, 24971, 37085, 4956, 7304, 51350, 7653, 59872, 34724, 1491, 58875, 7653, 54668, 44182, 7653, 34724, 17157, 22721, 26590, 47550, 7653, 57292, 17157, 3465, 58067, 69183, 7653, 54762, 44182, 7653, 59902, 59999, 17157, 5938, 48328, 19584, 56430, 48936, 21315, 32446, 36813, 7653, 49033, 15965, 11787, 390, 58067, 10354, 66162, 36969, 7653, 20503, 151, 1491, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [24]:
# check model output:
model_1 = WSDModel_approach1(len(vocab), 32, 32, len(labels), pad_index=vocab['PAD'])

for i, batch in enumerate(test_iter):
    gold_labels = [example[0] for example in batch]
    indices = [example[1] for example in batch]
    input_seqs = torch.Tensor([example[2] for example in batch]).long()
    output = model_1(input_seqs)
    print('gold_labels:', gold_labels)
    print('indices:', indices)
    print('input_seqs.shape:', input_seqs.shape)
    print('output.shape:', output.shape)  # predictions[0][3].shape == torch.Size([224]) (so for each batch and timestep there is a distribution over labels.)
  # Only look at first batch.
    break

gold_labels: [168, 65, 104, 68, 211, 72, 62, 132]
indices: [53, 26, 54, 43, 35, 29, 56, 19]
input_seqs.shape: torch.Size([8, 131])
output.shape: torch.Size([8, 131, 224])


In [96]:
from tqdm.auto import tqdm 

# Train the first model:

model_1 = WSDModel_approach1(len(vocab), 32, 32, len(labels), pad_index=vocab['PAD']).to(device)
loss_function = nn.CrossEntropyLoss(ignore_index=model_1.pad_index).to(device)
optimizer = optim.Adam(model_1.parameters(), lr=learning_rate)


for epoch in tqdm(range(epochs)):
    total_loss = 0
    for i, batch in enumerate(train_iter):
        
        # format of examples:
        # (self.int_senses[idx], self.indices[idx], self.int_contexts[idx])
        
        gold_labels = torch.Tensor([example[0] for example in batch]).long().to(device)
        indices = [example[1] for example in batch]
        input_seqs = torch.Tensor([example[2] for example in batch]).long().to(device)
        output = model_1(input_seqs)
        model_predictions = torch.stack([output[j][indices[j]] for j in range(len(batch))])
        
        loss = loss_function(model_predictions, gold_labels)
        total_loss += loss.item()
        
        # print average loss for the epoch
        print(f'epoch {epoch},', round(total_loss / (i + 1), 4), end='\r') 
        
        # compute gradients
        loss.backward()
        
        # update parameters
        optimizer.step()
        
        # reset gradients
        optimizer.zero_grad()


# Pickle trained model
pickle.dump(model_1, open('model_1.pickle', 'wb'))


# test model after all epochs are completed
accuracies = []
for batch in test_iter:
    gold_labels = [example[0] for example in batch]
    indices = [example[1] for example in batch] 
    input_seqs = torch.Tensor([example[2] for example in batch]).long().to(device)
    output = model_1(input_seqs).cpu().numpy(force=True)
    model_predictions = [np.argmax(output[i][indices[i]]) for i in range(len(batch))]
    batch_accuracies = [int(gold_labels[i] == model_predictions[i]) for i in range(len(batch))]
    accuracies.extend(batch_accuracies)

print('Accuracy: {}'.format(sum(accuracies)/len(accuracies)))


  0%|                                                                                             | 0/3 [00:00<?, ?it/s]

 33%|████████████████████████████▎                                                        | 1/3 [00:26<00:52, 26.25s/it]

 67%|████████████████████████████████████████████████████████▋                            | 2/3 [00:52<00:26, 26.24s/it]

100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:18<00:00, 26.22s/it]

Accuracy: 0.5490466798159106


In [ ]:
# Train the second model:

# 2.2 Running a transformer for WSD

In this section of the lab you'll try out the transformer, specifically the BERT model. For this we'll use the huggingface library (https://huggingface.co/).

You can find the documentation for the BERT model here (https://huggingface.co/transformers/model_doc/bert.html) and a general usage guide here (https://huggingface.co/transformers/quickstart.html).

What we're going to do is *fine-tune* the BERT model, i.e. update the weights of a pre-trained model. That is, we have a model that is trained on language modeling, but now we apply it to word sense disambiguation with the word representations it learnt from language modeling.

We'll use the same data splits for training and testing as before, but this time you'll not use a torchtext dataloader. Rather now you create an iterator that collects N sentences (where N is the batch size) then use the BertTokenizer to transform the sentence into integers. For your dataloader, remember to:
* Shuffle the data in each batch
* Make sure you get a new iterator for each *epoch*
* Create a vocabulary of *sense-labels* so you can calculate accuracy 

We then pass this batch into the BERT model and train as before. The BERT model will encode the sentence, then we send this encoded sentence into a prediction layer (you can either use the sentence-representation from bert, or the ambiguous word) like before and collect sense predictions.

About the hyperparameters and training:
* For BERT, usually a lower learning rate works best, between 0.0001-0.000001.
* BERT takes alot of resources, running it on CPU will take ages, utilize the GPUs :)
* Since BERT takes alot of resources, use a small batch size (4-8)
* Computing the BERT representation, make sure you pass the mask

**[10 marks]**

In [ ]:
def dataloader_for_bert(path_to_file, batch_size):
    ...

In [ ]:
class BERT_WSD(nn.Module):
    def __init__(self, ...):
        # your code goes here
        self.bert = ...
        self.classifier = ...
    
    def forward(self, batch):
        # your code goes here
        
        return predictions

In [ ]:
loss_function = ...
optimizer = ...
model = ...

for _ in range(epochs):
    # train model
    ...
    
# test model after all epochs are completed

# 3. Evaluation

Explain the difference between the first and second approach. What kind of representations are the different approaches using to predict word-senses? **[4 marks]**

Evaluate your model with per-word-form *accuracy* and comment on the results you get, how does the model perform in comparison to the baseline, and how do the models compare to each other? 

Expand on the evaluation by sorting the word-forms by the number of senses they have. Are word-forms with fewer senses easier to predict? Give a short explanation of the results you get based on the number of senses per word.

**[6 marks]**

How does the LSTMs perform in comparison to BERT? What's the difference between representations obtained by the LSTMs and BERT? **[2 marks]**

What could we do to improve our LSTM word sense disambiguation models and our BERT model? **[4 marks]**

# Readings

[1] Kågebäck, M., & Salomonsson, H. (2016). Word Sense Disambiguation using a Bidirectional LSTM. arXiv preprint arXiv:1606.03568.

[2] https://cl.lingfil.uu.se/~nivre/master/NLP-LexSem.pdf

## Statement of contribution

Briefly state how many times you have met for discussions, who was present, to what degree each member contributed to the discussion and the final answers you are submitting.

## Marks

This assignment has a total of 46 marks.